<a href="https://colab.research.google.com/github/Followb1ind1y/D2L_Pytorch_Study_Notes/blob/main/06_D2L_Builders_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dive into Deep Learning 中文学习笔记** 
# **6. 深度学习计算 （Builders’ Guide）**

## **6.1. 模型构造（Layers and Modules）**

为了实现一些复杂的网络，我们引入了神经网络 **模块（*Module*）** 的概念。一个模块可以描述一个单一的层，一个由多个层组成的组件，或者整个模型本身。使用模块抽象的一个好处是，它们可以被组合成更大的网络，并且通常是递归的。

从编程的角度来看，一个模块由一个 **类（*Class*）** 来代表。它的任何子类必须定义一个前向传播方法，将其输入转化为输出，并且必须存储任何必要的参数。请注意，有些模块根本不需要任何参数。最后，一个模块必须拥有一个反向传播方法，用于计算梯度。

我们重温一下之前用来实现 MLPs 的代码。下面的代码生成了一个具有 256 个单元的全连接隐藏层和 ReLU 激活的网络，然后是一个具有 10个 单元的全连接输出层（没有激活函数）。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])


在这个例子中，我们通过实例化 `nn.Sequential` 来构建我们的模型，并将各层的执行顺序作为参数传递。简而言之，`nn.Sequential` 定义了一种特殊的 `Module`，即在 PyTorch 中展示模块的类。它维护着一个有序的组成模块的列表。请注意，两个完全连接的层中的每一个都是 `Linear` Class 的实例，它本身就是 `Module` 的子类。前向传播（forward）方法也非常简单：它将列表中的每个模块连在一起，将每个模块的输出作为输入传给下一个模块。

### **6.1.1. 自定义 Module（A Custom Module）**

在我们实现自己的自定义模块之前，我们简要地总结一下每个模块必须提供的基本功能：

1. 取得输入数据作为其前向传播方法的参数。

2. 通过让前向传播方法返回一个值来生成一个输出。请注意，输出可能有与输入不同的形状。

3. 计算其输出相对于输入的梯度，这可以通过其反向传播方法获得。

4. 存储并提供对那些执行前向传播计算所需的参数。

5. 根据需要初始化模型参数。

In [3]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

`Module` 类是 `nn` 模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承 `Module` 类构造本节开头提到的多层感知机。这里定义的 MLP 类重载了 `Module` 类的 `__init__` 函数和 `forward` 函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

以上的 MLP 类中无须定义反向传播函数。系统将通过**自动求梯度**而自动生成反向传播所需的 `backward` 函数。

我们可以实例化 MLP 类得到模型变量 `net`。下面的代码初始化 `net` 并传入输入数据 `X` 做一次前向计算。其中，`net(X)` 会调用 MLP 继承自 `Module` 类的 `__call__` 函数，这个函数将调用 MLP 类定义的 `forward` 函数来完成前向计算。

In [6]:
net = MLP()
print(net(X).shape)
print(net(X))

torch.Size([2, 10])
tensor([[-0.0542,  0.4686, -0.0286, -0.0396, -0.1683,  0.2483, -0.0951, -0.1815,
         -0.1284, -0.0661],
        [ 0.0286,  0.3326,  0.0296, -0.0600, -0.2222,  0.1332, -0.1309, -0.1670,
         -0.0838,  0.0571]], grad_fn=<AddmmBackward0>)


注意，这里并没有将 `Module` 类命名为 `Layer`（层）或者 `Model`（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如 PyTorch 提供的 `Linear` 类），又可以是一个模型（如这里定义的 MLP 类），或者是模型的一个部分。

### **6.1.2. Sequential 类（The Sequential Module）**

当模型的前向计算为简单串联各个层的计算时，`Sequential` 类可以通过**更加简单**的方式定义模型。这正是 `Sequential` 类的目的：它可以接收一个子模块的有序字典（`OrderedDict`）或者一系列子模块作为参数来逐一添加 `Module` 的实例，而模型的前向计算就是将这些实例**按添加的顺序逐一计算**。

为了建立我们自己的简化 `MySequential`，我们只需要定义两个关键方法：1. 一个将模块逐一追加到列表中的方法。2. 一个前向传播的方法，将一个输入通过模块链，以它们被附加的相同顺序传递。下面的 `MySequential` 类提供了与默认 `Sequential` 类相同的功能。

In [8]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

在 `__init__` 方法中，我们通过调用 `add_modules` 方法添加每个模块。这些模块可以在以后被 children 方法访问。通过这种方式，系统知道所添加的模块，它将正确地初始化每个模块的参数。

当我们的 `MySequential` 的前向传播方法被调用时，每个添加的模块都会按照它们被添加的顺序执行。现在我们可以使用我们的 `MySequential` 类重新实现一个 MLP。

In [9]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

### **6.1.3. 在前向传播法中执行代码（Executing Code in the Forward Propagation Method）**

虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义 `forward` 函数，但直接继承 `Module` 类可以极大地拓展模型构造的灵活性。然而，有时我们可能想加入一些既不是前几层的结果也不是可更新参数的部分。我们称这些为 **恒定参数 (*constant parameters*)**。例如，我们想要一个计算函数 $f(x,w)=c \cdot w^{T}x$ 的层，其中 $x$ 输入，$w$ 是我们的参数，并且 $c$ 是一些指定的常数，在优化过程中不被更新。所以我们实现一个 `FixedHiddenMLP` 类，如下所示:

In [17]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个 `FixedHiddenMLP` 模型中，我们实现了一个隐藏层，其权重（`self.rand_weight`）在实例化时被随机初始化，此后保持不变。这个权重不是一个模型参数，因此它永远不会被反向传播所更新。然后，网络将这个 "固定 "层的输出通过一个全连接层。

In [18]:
net = FixedHiddenMLP()
net(X)

tensor(0.5330, grad_fn=<SumBackward0>)

我们可以混合和匹配各种方式将模块组装在一起:

In [19]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.0652, grad_fn=<SumBackward0>)

## **6.2. 参数管理（Parameter Management）**

一旦我们选择了一个架构并设置了超参数，我们就进入训练循环，我们的目标是找到使损失函数最小的参数值。训练结束后，我们将需要这些参数，以便进行未来的预测。此外，我们有时希望提取这些参数，以便在其他情况下重新使用它们，将我们的模型保存在磁盘上，以便在其他软件中执行，或者用于检查。我们首先来具有一个隐藏层的 MLP 例子。

In [20]:
import torch
from torch import nn

net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

### **6.2.1. 参数访问（Parameter Access）**

当一个模型通过 `Sequential` 类定义时，我们可以首先通过对模型的索引来访问任何一层，就好像它是一个列表。每个层的参数都很方便地位于其属性中。我们可以按以下方式检查第二个全连接层的参数：

In [21]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.3025,  0.3420,  0.2825, -0.0193, -0.3081,  0.3457, -0.1775,  0.0029]])),
             ('bias', tensor([-0.2881]))])

**6.2.1.1. 目标参数（Targeted Parameters）**

请注意，每个参数都被表示为 `parameter` 类的一个实例。要使用参数，我们首先需要访问底层的数值。有几种方法可以做到这一点。下面的代码从第二个神经网络层中提取偏差，返回一个参数类实例，并进一步访问该参数的值。

In [22]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.2881]))

`Parameter` 是复杂的对象，包含值、梯度和附加信息。这就是为什么我们需要明确地请求该值。除了值之外，每个参数还允许我们访问梯度。因为我们还没有为这个网络调用反向传播，它处于初始状态。

In [23]:
net[2].weight.grad == None

True

**6.2.1.2. 访问所有参数（All Parameters at Once）**

对于 `Sequential` 实例中含模型参数的层，我们可以通过 `Module` 类的 `parameters()` 或者 `named_parameters` 方法来访问所有参数（以迭代器的形式返回），后者除了返回参数 Tensor 外还会返回其名字。

In [24]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

### **6.2.2. 捆绑参数（Tied Parameters）**

通常情况下，我们想在多个层之间共享参数。下面，我们分配了一个全连接层，然后专门使用它的参数来设置另一个层的参数。这里我们需要在访问参数之前运行前向传播 `net(X)`。

In [25]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
